# 타입B 태그안뽑기

In [1]:
import re
from tqdm.notebook import tnrange
import csv
import json
from os import walk
import os
import pandas as pd
import datetime

In [2]:
# 파일 불러오기
def get_csv(input_repo, input_filename):
    result = pd.read_csv(f'{input_repo}/{input_filename}')
    
    return result

In [3]:
# 전체 문장에서 태그 안만 뽑음
# <=법률어휘 ㅇㅇㄴㅁ
def save_tag(sentence):
    regex = re.compile('(?<=\<{3}).*?(?=\>{3})')
    result = regex.findall(sentence)
    
    return result

In [4]:
# 태그 내 문장으로부터 법률어휘 추출
def draw_word(sentence_tagremoved):
    regex = re.compile('(?<=\<=).*(?=\>)')
    result = regex.search(sentence_tagremoved)
    
    return result

In [5]:
def get_small_tag_removed(sentence):
    keyword_list = save_tag(sentence)
    regex_small_tag = re.compile('(?=\<={1}).*?(?<=\>{1})')
    small_tag_removed = []
    small_tag = []
    for i in range(len(keyword_list)):
        small_tag.append(regex_small_tag.findall(keyword_list[i]))
        small_tag_removed.append(keyword_list[i])

        for j in range(len(small_tag[i])):
            small_tag_removed[i] = small_tag_removed[i].replace(small_tag[i][j], '')
            
    return small_tag_removed

In [6]:
def get_small_tag_removed_throw_morethan2_org(sentence):
    keyword_list = save_tag(sentence)
    regex_small_tag = re.compile('(?=\<={1}).*?(?<=\>{1})')
    regex_small_tag_org = re.compile('(?<=\<={1}).*')

    small_tag_removed_org = []
    small_tag_list_org = []

    for i in range(len(keyword_list)):
        small_tag = regex_small_tag.findall(keyword_list[i])
        if len(small_tag)<2:
            try:
                small_tag_list_org = regex_small_tag_org.search(keyword_list[i]).group()
                small_tag_removed_org.append(small_tag_list_org.replace('>', ''))
            except:
                small_tag_removed_org.append(keyword_list[i])


        else:
            keyword_list[i] = None

    for i in range(len(keyword_list)-1, -1, -1):
        if keyword_list[i] == None:
            del keyword_list[i]

    return small_tag_removed_org

In [7]:
def get_small_tag_removed_throw_morethan2(sentence):
    keyword_list = save_tag(sentence)
    regex_small_tag = re.compile('(?=\<={1}).*?(?<=\>{1})')
    regex_small_tag_sub = re.compile('(?=\<={1}).*')
    small_tag_removed = []
    small_tag_list = []

    for i in range(len(keyword_list)):
        small_tag = regex_small_tag.findall(keyword_list[i])
        small_tag_sub = regex_small_tag_sub.findall(keyword_list[i])
        if len(small_tag)<2:
            try:           # 정상적인 경우
                small_tag_list.append(regex_small_tag.search(keyword_list[i]).group())
                small_tag_removed.append(keyword_list[i].replace(small_tag[0], ''))
            except:
                try:       # =>>>> 가 나올 때
                    small_tag_list.append(regex_small_tag_sub.search(keyword_list[i]).group())
                    small_tag_removed.append(keyword_list[i].replace(small_tag_sub[0], ''))
                except:    # <<<>>> 안에 <=> 아예 없을 때
                    small_tag_removed.append(keyword_list[i])

        else:
            keyword_list[i] = None

    for i in range(len(keyword_list)-1, -1, -1):
        if keyword_list[i] == None:
            del keyword_list[i]

    return small_tag_removed

In [8]:
# csv형태로 저장
def save_as_csv(input_repo, output_repo, result):
    year = str(datetime.datetime.now().year)[2:]
    month = str(datetime.datetime.now().month)
    day = str(datetime.datetime.now().day)
    hour = str(datetime.datetime.now().hour)
    minute = str(datetime.datetime.now().minute)

    print(f'{output_repo}의 경로에 최종_{year}{month}{day}_{hour}{minute}.csv의 이름으로 파일 생성이 완료되었습니다.')

    result.to_csv(f'{output_repo}/최종_{year}{month}{day}_{hour}{minute}.csv', index=False, encoding='utf-8-sig')

In [12]:
input_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/TypeB태그안뽑기'
output_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/TypeB태그안뽑기/TypeB 최종'

input_filename = '태그안뽑기_211117_1618.csv'    # 바꿔

In [13]:
org = get_csv(input_repo, input_filename)

In [14]:
result_transcription_org = []
for i in tnrange(len(org)):
    result_temp = get_small_tag_removed_throw_morethan2_org(str(org['typeA'][i]))
    result_transcription_org.append(result_temp)

  0%|          | 0/76870 [00:00<?, ?it/s]

In [15]:
result_transcription = []
for i in tnrange(len(org)):
    result_temp = get_small_tag_removed_throw_morethan2(str(org['transcription'][i]))
    result_transcription.append(result_temp)

  0%|          | 0/76870 [00:00<?, ?it/s]

In [ ]:
result = pd.DataFrame(columns=['법률 용어', '쓰임새', '번역예시', '키워드'])
result['쓰임새'] = []
result['번역예시'] = []
for i in tnrange(len(org)):
    word = org['word'][i]
    for j in range(len(result_transcription_org[i])):
        word_org = result_transcription_org[i][j]
        try:
            trans = result_transcription[i][j]
        except:
            trans = None
        keyword = None
        if (word in word_org)&(word_org != word)&(trans!=None):
                result = result.append({'법률 용어':word, '쓰임새':word_org, '번역예시':trans, '키워드':keyword}, ignore_index=True)
        else:
            pass

  0%|          | 0/76870 [00:00<?, ?it/s]

In [ ]:
save_as_csv(input_repo, output_repo, result)

In [18]:
# nan값 처리해야함. str로 넣어놨음